Environment of the agent comprises of a map in graph form with vertices represented by cities and edges represented by distance between cities. 
Assumptions:
1. We are ignoring factors such as traffic, road conditions, accidents etc. which can influence time taken to cover distance. Hence distance is the only factor used to decide the optimal path.
2. We are ignoring exact starting location within Panji city and exact destination in Chennai city. City center is assumed as point of start and end.
Characteristics of the environment:
1. Partially observable - since agent can only see the neighboring cities when it is at a particular city. 
2. Static - since we are only considering distance between cities and connections between cities. Environment doesn’t change while agent is deliberating.
3. Sequential - since agent needs to know sequence of previous steps while deciding the next step.
4. Deterministic - since result of agent's action is completely determined based on current state and action taken by agent.
5. Discrete - finite number of states involved and discrete set of percepts and actions.
6. Single agent - since only one agent is involved.


We have created following objects to help us represent the environment:
1. City: We have created class City with attributes cityName, latitude, longitude and adjacent neighbors along with their respective distances. Python dictionary data structure is used to store adjacent cities. Key in this dictionary is city name and value is distance to that city.

Example Panji city:

{ cityName = 'Panji',

  latitude = 15.4909,
  
  longitude = 73.8278,
  
  adjacent = { 'Mangalore'=365, 'Bellari' = 409, 'Raichur'=457}
  
 }

In [1]:
class City:
    def __init__(self, cityName, lat, long):
        self.cityName = cityName
        self.latitude = lat
        self.longitude = long
        self.adjacent = {}

    def get_cityName(self):
        return self.cityName
    
    def get_lat(self):
        return self.latitude
    
    def get_long(self):
        return self.longitude   

    def get_distance(self, neighbor):
        return self.adjacent[neighbor]
    
    def add_neighbor(self, neighbor, distance=0):
        self.adjacent[neighbor] = distance

    def get_connections(self):
        return self.adjacent.keys()  

2. Environment: Class Environment is used to represent the environemnt. This class contains a dictionary called cities which has city objects. It also provides functions to add city, get city as well as get all neighbors of a city.
Below is an example to depict how this data structure looks like.

cities=

{ 'Panji':
     
     { cityName = 'Panji',

      latitude = 15.4909,
  
      longitude = 73.8278,
  
      adjacent = { 'Mangalore'=365, 'Bellari' = 409, 'Raichur'=457}
  
     },
     
'Mangalore':
     
     { cityName = 'Mangalore',

      latitude = 12.9141,
  
      longitude = 74.8560,
  
      adjacent = { 'Panji'=365, 'Bangalore' = 352, 'Kozhicode'=233}
  
     }
     ...
     
     ...
     
}

In [2]:
class Environment:
    def __init__(self):
        self.cities = {}


    def add_city(self, node, lat, long):
        new_city = City(node, lat, long)
        self.cities[node] = new_city
        return new_city

    def get_city(self, n):
        if n in self.cities:
            return self.cities[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        self.cities[frm].add_neighbor(self.cities[to], cost)
        self.cities[to].add_neighbor(self.cities[frm], cost)

    def get_adjacent(self,n):
        return n.get_connections()

Define the haversine formula and what it does

#haversine formula representation 

haversine formula calculates the great-circle distance between two points – that is, the shortest distance over the earth’s surface between the points (ignoring any hills, fly over, etc).

Haversine formula:

a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
c = 2 ⋅ atan2( √a, √(1−a) )
d = R ⋅ c

where	φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
note that angles need to be in radians to pass to trig functions!


2) Define a function which calculates the heuristic distance from each city to the destination city in the following code block

In [3]:
#Function for calculating distance from each node/city to Destination using latitude and longitude.
import math
def huristic_dist(origin, destination):
    radius = 6371 #Based on Haversine formula assumption of mean radius in km
    latitude1 = origin.get_lat()
    longitude1 = origin.get_long()
    latitude2 = destination.get_lat()
    longitude2 = destination.get_long()
    

    diffLat = math.radians(latitude2-latitude1)
    diffLon = math.radians(longitude2-longitude1)
    a = math.pow(math.sin(diffLat/2),2) + math.cos(math.radians(latitude1)) * math.cos(math.radians(latitude2)) * math.pow(math.sin(diffLon/2),2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

3) Implementation of A* Algorithm . Feel free to add code blocks for each methods needed starting here.
Please modularize the implementation of A* and write each of them in a code block. 

In [9]:
class Agent:
    visitedCities= set()
    frontierCities= set()
    f = {} # stores f(n); f(n) = g(n)+h(n)
    g = {} # stores g(n), the actual cost from start to a node
    # to retrieve the best once algorithm completes
    previousOptimalNode = {}
    finalPath=[]
    finalCost=0
    citiesVistedCount=0
        
    def selectBestCityFromFrontier(self):
        current_cityName = None
        currentFscore = None        
        for pos in self.frontierCities:
            if current_cityName is None or self.f[pos] < currentFscore:
                currentFscore = self.f[pos]
                current_cityName = pos
        return current_cityName,currentFscore 

    def generate(self,environment,current_cityName):
        current = environment.get_city(current_cityName)
        for neighbor in environment.get_adjacent(current):
            neighbor_cityName = neighbor.get_cityName()
            if neighbor_cityName in self.visitedCities:
                continue
            new_g_for_neighbor = self.g[current_cityName] + current.get_distance(neighbor)
            if neighbor_cityName in self.frontierCities:
                if new_g_for_neighbor >= self.g[neighbor_cityName]:
                    continue
            else:
                self.frontierCities.add(neighbor_cityName)
                self.citiesVistedCount+=1
                print(neighbor_cityName)
            self.previousOptimalNode[neighbor_cityName] = current_cityName
            self.g[neighbor_cityName] = new_g_for_neighbor
            self.f[neighbor_cityName] = self.g[neighbor_cityName] + huristic_dist(neighbor,end)   
            
    def backtrackPath(self,start_cityName,end_cityName):
        self.finalPath.append(end_cityName)
        curr_cityName = end_cityName 
        while curr_cityName != start_cityName:
            curr_cityName = self.previousOptimalNode[curr_cityName]
            self.finalPath.append(curr_cityName)
        self.finalPath.reverse()
        self.finalCost=self.g[end_cityName]        
        
    def AStar(self,start,end,environment):
        start_cityName = start.get_cityName()
        end_cityName = end.get_cityName()
        
        #   Add start to frontier
        self.frontierCities.add(start_cityName)
        self.citiesVistedCount+=1
        print(start_cityName)
        self.g[start_cityName] = 0
        self.f[start_cityName] = huristic_dist(start,end) + self.g[start_cityName]

        #2. Loop through frontier until soln is found or frontier empty
        while len(self.frontierCities) > 0:
            # select best node from frontier
            current_cityName,currentFscore  = self.selectBestCityFromFrontier()            

            #4. check if we have reached goal  
            if current_cityName == end_cityName:
                print('success')
                self.backtrackPath(start_cityName,end_cityName)
                return self.finalPath, self.finalCost, self.citiesVistedCount

            #5. mark current vertex as closed
            self.frontierCities.remove(current_cityName)
            self.visitedCities.add(current_cityName)
            

            #6. Expand current node and add all childs to frontier
            self.generate(environment,current_cityName)

        print("no solution")

Call your main function/algorithm block in the next code block with appropriate input representation

In [10]:
e = Environment()

e.add_city('Panji',15.4909,73.8278)
e.add_city('Raichur',16.2076,77.3463)
e.add_city('Kurnool',15.8281,78.0373)
e.add_city('Tirupati',13.6288,79.4192)
e.add_city('Bellari',15.1394, 76.9214)
e.add_city('Mangalore',12.9141,74.8560)
e.add_city('Bangalore',12.9716,77.5946)
e.add_city('Kozhicode',11.2588,75.7804)
e.add_city('Nellore',14.4426,79.9865)
e.add_city('Chennai',13.0827, 80.2707)


e.add_edge('Panji', 'Raichur', 457)  
e.add_edge('Panji', 'Bellari', 409)
e.add_edge('Panji', 'Mangalore', 365)
e.add_edge('Raichur', 'Tirupati', 453)
e.add_edge('Raichur', 'Kurnool', 100)
e.add_edge('Tirupati', 'Bellari', 379)
e.add_edge('Tirupati', 'Chennai', 153)
e.add_edge('Tirupati', 'Nellore', 136)
e.add_edge('Bangalore', 'Kozhicode', 356)
e.add_edge('Bangalore', 'Chennai', 346)
e.add_edge('Bangalore', 'Bellari', 311)
e.add_edge('Bangalore', 'Mangalore', 352)
e.add_edge('Kozhicode', 'Mangalore', 233)
e.add_edge('Tirupati', 'Kurnool', 340)
e.add_edge('Nellore', 'Kurnool', 325)
e.add_edge('Nellore', 'Chennai', 175)

start=e.get_city('Panji')
end=e.get_city('Chennai')
a = Agent()

path,cost, citiesVistedCount = a.AStar(start,end,e)


Panji
Raichur
Bellari
Mangalore
Tirupati
Bangalore
Chennai
Nellore
Kurnool
success


The agent should provide expected output for questions mentioned below in the subsequent blocks

(3.1) Path taken to reach destination from Panaji

In [6]:
# Execute statement to retrieve the path taken here
print('Best path from Panji to Chennai:'+str(path))

Best path from Panji to Chennai:['Panji', 'Bellari', 'Tirupati', 'Chennai']


(3.2) Cost of the path

In [7]:
# Execute statement to retrieve the cost of the path here
print('cost of best path to Chennai:'+str(cost))

cost of best path to Chennai:941


(3.3) Total Number of nodes vistied to get this state

In [8]:
# Execute statement to retrieve the total number of nodes visited to get this state here
print('total nodes visted to reach Chennai:'+str(len(path)))
print('total intermediate cities visited to reach Chennai from Panji:'+str(len(path)-2))
print('total nodes visted to come up with solution:'+str(citiesVistedCount))


total nodes visted to reach Chennai:4
total intermediate cities visited to reach Chennai from Panji:2
total nodes visted to come up with solution:9
